############################################
# User-Based Collaborative Filtering
#############################################

# Adım 1: Veri Setinin Hazırlanması
# Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
# Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
# Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi
# Adım 5: Weighted Average Recommendation Score'un Hesaplanması
# Adım 6: Çalışmanın Fonksiyonlaştırılması


In [ ]:
#############################################
# Adım 1: Veri Setinin Hazırlanması
#############################################
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
#Dataframe görüntüleme ayarları

In [ ]:
#İki dateframe çekiliyor, birleştiriliyor, 1000'den az yorum alanlar eleniyor. 
#gözlemler:kullanıcı id, değişkenler:film adı, kesişimler:puan
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
    rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()

In [ ]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values) 
#random_state her zaman aynı örneği vermesini sağlıyor

In [ ]:
random_user #28941


In [ ]:
#############################################
# Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
#############################################
random_user_df = user_movie_df[user_movie_df.index == random_user]
#random_user_df dataframe'deki ilgili kullanıcının olduğu gözlemi verir. tek satırlık bir dataframe oluşur.

In [ ]:
#ilgili kullanıcıların izlediği filmlerden oluşan bir liste oluşturulur.
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

In [ ]:
len(movies_watched) #33 film izlemiş

In [ ]:
#ilgili kullanıcının merak ettiğimiz bir filmi izleyip izlemediğine dair sorgu;
user_movie_df.loc[user_movie_df.index == random_user,
                  user_movie_df.columns == "Silence of the Lambs, The (1991)"]


In [ ]:
#############################################
# Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
#############################################

#Dataframe tüm kullanıcılar var, değişken olarak sadece ilgili kullanıcının izlediği filmler var.
movies_watched_df = user_movie_df[movies_watched]

In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()
#Transpouzunu aldığımızda, 33 tane film gözlem haline gelir 
#ve yaklaşık 138bin kullanıcı sütun adı olarak değişeknlere dönüşür
#Her değişkenin boş olmayan değerlerini topla dediğimizde, her kullanıcının kaç adet film izlediğini buluyoruz.
""" user_movie_count out:
userId
1.0          1
2.0          2
3.0          4
4.0          6
5.0         11
            ..
138489.0     1
138490.0     7
138491.0     0
138492.0     2
138493.0     9
Length: 138493, dtype: int64
"""

In [ ]:
#indexte bulunan kullanıcı adlarını farklı sütuna taşıyor. indexleri yerleştiriyor.
user_movie_count = user_movie_count.reset_index()

In [ ]:
#değişken adlarını isimlendiriyoruz
user_movie_count.columns = ["userId", "movie_count"]

In [ ]:
#filtreleme işlemi
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)

In [ ]:
#inceleme
user_movie_count[user_movie_count["movie_count"] == 33].count()

In [ ]:
#benzer filmler izleyen kullanıcıların listesini elde etme;
perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

In [ ]:
users_same_movies


# Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi


Bunun için 3 adım gerçekleştireceğiz:

**1. Sinan ve diğer kullanıcıların verilerini bir araya getireceğiz.**

**2. Korelasyon df'ini oluşturacağız.**
 
 **3. En benzer bullanıcıları (Top Users) bulacağız**

In [ ]:
# Bunun için 3 adım gerçekleştireceğiz:
# 1. Sinan ve diğer kullanıcıların verilerini bir araya getireceğiz.
# 2. Korelasyon df'ini oluşturacağız.
# 3. En benzer bullanıcıları (Top Users) bulacağız

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

In [ ]:
final_df.head()

In [ ]:
#her kullanıcının, diğer tüm kullanıcıları olan korelasyonlarını sıralayacak şekilde yeni dataframe oluşturma komutu.
corr_df = final_df.T.corr().unstack()
corr_df
"""out
userId   userId  
91.0     91.0        1.000000
         130.0       0.379803
         156.0       0.320108
         158.0       0.660170
         160.0       0.516446
                       ...   
28941.0  138279.0    0.426883
         138382.0   -0.045278
         138415.0    0.481604
         138483.0    0.567603
         28941.0     1.000000
Length: 17139600, dtype: float64
"""

In [ ]:
pd.set_option('display.max_rows', 200)
"""#her kullanıcının, diğer tüm kullanıcıları olan korelasyonlarını sıralayacak şekilde yeni dataframe oluşturma komutu.
corr_df = final_df.T.corr().unstack()"""

#corr() değişkenlerin birbirine olan benzerliğini gösterir. 
#hem gözlemler hem değişkenler kullanıcı id olur. kesişimlerde korelasyon değerleri olur.
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()

corr_df = pd.DataFrame(corr_df, columns=["corr"])

corr_df.index.names = ['user_id_1', 'user_id_2']

corr_df = corr_df.reset_index()
corr_df.head()


In [ ]:
#hedef kullanıcımızla yüksek korelasyon sergileyen, en az 20 film izlemiş kullanıcıların listesi ve korelasyon puanı.
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr',ascending=False)
top_users

In [ ]:
#değişken adı değiştirilmesi
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

In [ ]:
#ulaştığımız listedeki insanların söz konusu verdiği puanları bulunduran bir dataframe oluştuldu. 19591 gözlem.
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings

In [ ]:
#random user olarak belirttiğimiz hedef kullanıcımızı çıakrtıyoruz;
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]

In [ ]:
top_users_ratings

In [ ]:
import pandas as pd
#############################################
# Adım 5: Weighted Average Recommendation Score'un Hesaplanması
#############################################
#korelasyon ağırlığında puanlar skora dönüştürülüyor
top_users_ratings["weighted_rating"] = top_users_ratings["corr"]* top_users_ratings["rating"]
top_users_ratings.head() #19558row x 5 columns

#Burada dikkat etmemiz gereken nokta; bu sokorlar normal puanların hep altında oalcaktır. 
#Temsilen skor 3.6 ise kullanıcımızın tahminen 4.0 puan vereceği gibi bir tahminde bulunabiliriz.

In [ ]:
#seçtiğimiz benzeşen kullanıcıların, her filme verdiği puanların, korelasyon ağırlığında ortalama değeri, skoru
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"}) #5697 row

In [ ]:
#elde ettiğimiz filmlerin kullanıcımıza göre ayarlanmış skorlarından oluşan datarframe'i bir değişkene atadık.
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

#index oluşturuyoruz, index'deki kullanıcı id'leri 0. index'e gidiyor.
recommendation_df = recommendation_df.reset_index()

recommendation_df[["movieId"]].nunique()

#skoru belli bir düzeyin üstünde olan filmleri listede bırakıyoruz.
recommendation_df[recommendation_df["weighted_rating"] > 3.7]

#skoru belli bir düzeyin üstünde olan filmleri listede bırakıyoruz, skora göre sıralıyoruz, bu dataframe'i bir değişkene atıyoruz.
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.7].sort_values("weighted_rating", ascending=False)

#dataframe'de film adları da olsun diye movie.csv datarfame'i ile merge ediyoruz.
movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
movies_to_be_recommend.merge(movie[["movieId", "title"]])

In [ ]:

#############################################
# Adım 6: Çalışmanın Fonksiyonlaştırılması
#############################################

def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
    rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()

# perc = len(movies_watched) * 60 / 100
# users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]



def user_based_recommender(random_user, user_movie_df, ratio=60, cor_th=0.65, score=3.7):
    import pandas as pd
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    movies_watched_df = user_movie_df[movies_watched]
    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId", "movie_count"]
    perc = len(movies_watched) * ratio / 100
    users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]

    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                          random_user_df[movies_watched]])

    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns=["corr"])
    corr_df.index.names = ['user_id_1', 'user_id_2']
    corr_df = corr_df.reset_index()

    top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= cor_th)][
        ["user_id_2", "corr"]].reset_index(drop=True)

    top_users = top_users.sort_values(by='corr', ascending=False)
    top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
    rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')
    top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
    top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

    recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
    recommendation_df = recommendation_df.reset_index()

    movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > score].sort_values("weighted_rating", ascending=False)
    movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
    return movies_to_be_recommend.merge(movie[["movieId", "title"]])

random_user = int(pd.Series(user_movie_df.index).sample(1).values)
user_based_recommender(random_user, user_movie_df)

